In [1]:
import os, sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

import pytorch_lightning as pl
from codebase import lit_model,data, model,training

g:\Software\Anaconda\envs\pytorch_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
g:\Software\Anaconda\envs\pytorch_env\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: [WinError 126] The specified module could not be found
  warn(f"Failed to load image Python extension: {e}")


In [2]:
import torch

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
path = r"E:\Coding Environment\ocr\codebase\training"
curr_dir = os.getcwd()
os.chdir(path)
opt = training.get_config("config.yaml")
os.chdir(curr_dir)

In [5]:
data_obj = data.datamodule(opt)

g:\Software\Anaconda\envs\pytorch_env\lib\site-packages\pytorch_lightning\core\datamodule.py:122: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  "DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7."
g:\Software\Anaconda\envs\pytorch_env\lib\site-packages\pytorch_lightning\core\datamodule.py:141: LightningDeprecationWarning: DataModule property `val_transforms` was deprecated in v1.5 and will be removed in v1.7.
  "DataModule property `val_transforms` was deprecated in v1.5 and will be removed in v1.7."


In [31]:
data_obj.setup()
x, y = next(iter(data_obj.train_dataloader()))

g:\Software\Anaconda\envs\pytorch_env\lib\site-packages\pytorch_lightning\core\datamodule.py:470: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  f"DataModule.{name} has already been called, so it will not be called again. "


In [32]:
x.shape

torch.Size([4, 3, 64, 600])

In [33]:
y

('Ken-Telemadrid', '2-861 Cvd', "Warner } bar 'joke'", 'Nikkie')

In [12]:
model_obj = model.Model(opt)
#model_obj = model_obj.to(device)

In [7]:
lit_model_class = lit_model.BaseLitModel
lit_mode = lit_model_class(opt=opt, model=model_obj,args = None)

In [17]:
trainer = pl.Trainer( max_epochs = 2)

trainer.fit(lit_mode, datamodule=data_obj)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
g:\Software\Anaconda\envs\pytorch_env\lib\site-packages\pytorch_lightning\trainer\trainer.py:1585: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`."

  | Name      | Type     | Params
---------------------------------------
0 | model     | Model    | 3.8 M 
1 | loss_fn   | CTCLoss  | 0     
2 | train_acc | Accuracy | 0     
3 | val_acc   | Accuracy | 0     
4 | test_acc  | Accuracy | 0     
---------------------------------------
3.8 M     Trainable params
0         Non-trainable params
3.8 M     Total params
15.128    Total estimated model params size (MB)


Epoch 1:  18%|█▊        | 10/56 [00:29<02:17,  2.99s/it, loss=nan, v_num=1, validation/loss=nan.0]

g:\Software\Anaconda\envs\pytorch_env\lib\site-packages\pytorch_lightning\trainer\trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


Epoch 1:  18%|█▊        | 10/56 [00:40<03:06,  4.05s/it, loss=nan, v_num=1, validation/loss=nan.0]

In [8]:
log_dir = lit_mode.log_dirname()
logger = pl.loggers.TensorBoardLogger(log_dir)
experiment_dir = logger.log_dir

goldstar_metric = "validation/loss"
filename_format = "epoch={epoch:04d}-validation.loss={validation/loss:.3f}"
checkpoint_callback = pl.callbacks.ModelCheckpoint(
        save_top_k=5,
        filename=filename_format,
        monitor=goldstar_metric,
        mode="min",
        auto_insert_metric_name=False,
        dirpath=experiment_dir,
        every_n_epochs= 1, # can be passed as an argument
    )

summary_callback = pl.callbacks.ModelSummary(max_depth=2)

callbacks = [summary_callback, checkpoint_callback]
if opt.stop_early:
    early_stopping_callback = pl.callbacks.EarlyStopping(
            monitor="validation/loss", mode="min", patience=opt.stop_early
        )
    callbacks.append(early_stopping_callback)

trainer = pl.Trainer( callbacks=callbacks, logger=logger, accelerator = opt.accelerator, max_epochs = 25, devices = opt.devices)

trainer.fit(lit_mode, datamodule=data_obj)

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                    | Type                 | Params
-----------------------------------------------------------------
0 | model                   | Model                | 3.8 M 
1 | model.FeatureExtraction | VGG_FeatureExtractor | 1.4 M 
2 | model.AdaptiveAvgPool   | AdaptiveAvgPool2d    | 0     
3 | model.SequenceModeling  | Sequential           | 2.4 M 
4 | model.Prediction        | Linear               | 24.9 K
5 | loss_fn                 | CTCLoss              | 0     
6 | train_acc               | Accuracy             | 0     
7 | val_acc                 | Accuracy             | 0     
8 | test_acc                | Accuracy             | 0     
-----

Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

g:\Software\Anaconda\envs\pytorch_env\lib\site-packages\pytorch_lightning\trainer\data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


g:\Software\Anaconda\envs\pytorch_env\lib\site-packages\pytorch_lightning\utilities\data.py:60: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 32. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"
g:\Software\Anaconda\envs\pytorch_env\lib\site-packages\pytorch_lightning\trainer\data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
g:\Software\Anaconda\envs\pytorch_env\lib\site-packages\pytorch_lightning\trainer\data_loading.py:433: UserWarning: The number of training samples (28) is smaller than the logging interval 

Epoch 0:  50%|█████     | 28/56 [00:06<00:06,  4.27it/s, loss=91.3, v_num=8]

g:\Software\Anaconda\envs\pytorch_env\lib\site-packages\pytorch_lightning\utilities\data.py:60: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 10. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"


Epoch 24: 100%|██████████| 56/56 [00:09<00:00,  5.63it/s, loss=4.38, v_num=8, validation/loss=4.360]
